In [53]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv

# Specify the path to model config and checkpoint file
config_file = '/home/aiarhipov/centernet/exps/19_eval/config.py'
checkpoint_file = '/home/aiarhipov/centernet/exps/19_eval/epoch60_resnet34_batch2.pth'

# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')

# test a single image and show the results
# img_path = '/home/aiarhipov/centernet/imgs/0c38706d33642e2c68389688009a4c3958601299.jpg'  # or img = mmcv.imread(img), which will only load it once
img_path = '/home/aiarhipov/centernet/imgs/0d995a743dccbda88af4955cd07ab5bf3b8b663e.jpg'  # or img = mmcv.imread(img), which will only load it once
res = inference_detector(model, img_path)
result = res
# visualize the results in a new window
# model.show_result(img, result)
# or save the visualization results to image files
# model.show_result(img, result, out_file='result.jpg')
threshold = 0.5
show_result_pyplot(model, img_path, res, score_thr = threshold, out_file = "/home/aiarhipov/centernet/imgs/output.jpg")

2023-01-15 19:26:05,223 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.0.conv is upgraded to version 2.
2023-01-15 19:26:05,226 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.2.conv is upgraded to version 2.
2023-01-15 19:26:05,227 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.4.conv is upgraded to version 2.


load checkpoint from local path: /home/aiarhipov/centernet/exps/19_eval/epoch60_resnet34_batch2.pth


In [54]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler


boxes = result[0]
confident_boxes = boxes[boxes[:, 4] > threshold]
count_confident_boxes = confident_boxes.shape[0]

updated_confident_boxes=confident_boxes

X = confident_boxes[:, [0, 2]].flatten()
Y = confident_boxes[:, [1, 3]].flatten()

X_scaled = MinMaxScaler().fit_transform(X.reshape((-1, 1)))
Y_scaled = MinMaxScaler().fit_transform(Y.reshape((-1, 1)))

min_samples = int(np.sqrt(count_confident_boxes) / 2)
print("min_samples = ", min_samples)
clustering_X = DBSCAN(eps=0.01, min_samples=min_samples).fit_predict(X_scaled)
clustering_Y = DBSCAN(eps=0.01, min_samples=min_samples).fit_predict(Y_scaled)

labels_X = set(clustering_X)
labels_Y = set(clustering_Y)


img = Image.open(img_path)
draw = ImageDraw.Draw(img)
colors = [(255,0,0), (255,255,0), (255,0,255)]
width = 5

x_correct = []
y_correct = []
for x in labels_X:
    if x != -1:
        x_mean = int(np.mean(X[np.array(clustering_X) == x]))
        # print(f"x_label = {x}, x_mean = {x_mean}")
        x_correct.append(x_mean)
    
for y in labels_Y:
    if y != -1:
        y_mean = int(np.mean(Y[np.array(clustering_Y) == y]))
        # print(f"y_label = {y}, y_mean = {y_mean}")
        y_correct.append(y_mean)

for x in labels_X:
    if x != -1:
        x_mean = int(np.mean(X[np.array(clustering_X) == x]))
        endpoints = (x_mean, max(y_correct)), (x_mean, min(y_correct))
        draw.line(endpoints, fill=colors[0], width=width)
for y in labels_Y:
    if y != -1:
        y_mean = int(np.mean(Y[np.array(clustering_Y) == y]))
        endpoints = (max(x_correct), y_mean), (min(x_correct), y_mean)
        draw.line(endpoints, fill=colors[0], width=width)
    
print(f"sorted_confident_x = {sorted(x_correct)}")
print(f"sorted_confident_y = {sorted(y_correct)}")

img.save("/home/aiarhipov/centernet/imgs/output_lines.jpg")

min_samples =  3
sorted_confident_x = [0, 76, 293, 509, 725, 941, 1157, 1280]
sorted_confident_y = [170, 229, 285, 342, 399, 457, 513, 570, 628, 685]


In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler

def model_result_to_aligned_result(result, threshold):
    boxes = result[0]
    
    confident_boxes = boxes[boxes[:, 4] > threshold]
    count_confident_boxes = confident_boxes.shape[0]
    
    updated_confident_boxes = pd.DataFrame(confident_boxes, columns=["x1", "y1", "x2", "y2", "confidance"])
    print(updated_confident_boxes)
    X = confident_boxes[:, [0, 2]].flatten(order="F")
    Y = confident_boxes[:, [1, 3]].flatten(order="F")

    X_scaled = MinMaxScaler().fit_transform(X.reshape((-1, 1)))
    Y_scaled = MinMaxScaler().fit_transform(Y.reshape((-1, 1)))

    min_samples = int(np.sqrt(count_confident_boxes) / 2)
    print("min_samples = ", min_samples)
    clustering_X = DBSCAN(eps=0.01, min_samples=min_samples).fit_predict(X_scaled)
    clustering_Y = DBSCAN(eps=0.01, min_samples=min_samples).fit_predict(Y_scaled)

    labels_X = set(clustering_X)
    labels_Y = set(clustering_Y)
    
    updated_confident_boxes["label_x1"] = clustering_X[:count_confident_boxes]
    updated_confident_boxes["label_y1"] = clustering_Y[:count_confident_boxes]
    updated_confident_boxes["label_x2"] = clustering_X[count_confident_boxes:]
    updated_confident_boxes["label_y2"] = clustering_Y[count_confident_boxes:]

    # updated_confident_boxes["x1_upd"] = updated_confident_boxes.groupby(["label_x1"])["x1"].transform("mean")
    # updated_confident_boxes["y1_upd"] = updated_confident_boxes.groupby(["label_y1"])["y1"].transform("mean")
    # updated_confident_boxes["x2_upd"] = updated_confident_boxes.groupby(["label_x2"])["x2"].transform("mean")
    # updated_confident_boxes["y2_upd"] = updated_confident_boxes.groupby(["label_y2"])["y2"].transform("mean")
    
    # updated_confident_boxes["x1_upd"] = updated_confident_boxes["x1_upd"].astype(int)
    # updated_confident_boxes["y1_upd"] = updated_confident_boxes["y1_upd"].astype(int)
    # updated_confident_boxes["x2_upd"] = updated_confident_boxes["x2_upd"].astype(int)
    # updated_confident_boxes["y2_upd"] = updated_confident_boxes["y2_upd"].astype(int)
    
    updated_confident_boxes["x1_upd"] = 1
    updated_confident_boxes["y1_upd"] = 1
    updated_confident_boxes["x2_upd"] = 5
    updated_confident_boxes["y2_upd"] = 5
    
    img = Image.open(img_path)
    draw = ImageDraw.Draw(img)
    colors = [(255,0,0), (255,255,0), (255,0,255)]
    width = 5

    x_correct = []
    y_correct = []
    for x in labels_X:
        if x != -1:
            x_mean = int(np.mean(X[np.array(clustering_X) == x]))
            x_correct.append(x_mean)

            updated_confident_boxes["x1_upd"][updated_confident_boxes["label_x1"] == x] = x_mean
            updated_confident_boxes["x2_upd"][updated_confident_boxes["label_x2"] == x] = x_mean
    for y in labels_Y:
        if y != -1:
            y_mean = int(np.mean(Y[np.array(clustering_Y) == y]))
            y_correct.append(y_mean)

            updated_confident_boxes["y1_upd"][updated_confident_boxes["label_y1"] == y] = y_mean
            updated_confident_boxes["y2_upd"][updated_confident_boxes["label_y2"] == y] = y_mean
            
    # for x in labels_X:
    #     if x != -1:
    #         x_mean = int(np.mean(X[np.array(clustering_X) == x]))
    #         endpoints = (x_mean, max(y_correct)), (x_mean, min(y_correct))
    #         draw.line(endpoints, fill=colors[0], width=width)
    # for y in labels_Y:
    #     if y != -1:
    #         y_mean = int(np.mean(Y[np.array(clustering_Y) == y]))
    #         endpoints = (max(x_correct), y_mean), (min(x_correct), y_mean)
    #         draw.line(endpoints, fill=colors[0], width=width)
        
    # print(f"sorted_confident_x = {sorted(x_correct)}")
    # print(f"sorted_confident_y = {sorted(y_correct)}")

    # img.save("/home/aiarhipov/centernet/imgs/output_lines.jpg")
    return [updated_confident_boxes[["x1_upd", "y1_upd", "x2_upd", "y2_upd", "confidance"]].to_numpy()]

upd_res = model_result_to_aligned_result(result, threshold)

             x1          y1           x2          y2  confidance
0    292.159973  286.068909   510.660583  341.723938    0.889665
1    291.904541  341.950165   510.744263  399.858185    0.885304
2    292.007660  570.465271   511.048645  627.256409    0.871573
3    724.143433  630.468567   942.558960  683.702820    0.870791
4    725.313660  342.440369   942.441467  399.614441    0.870579
..          ...         ...          ...         ...         ...
58   939.884033  172.461670  1159.166748  229.938812    0.694375
59   507.686584  171.423981   726.961487  231.210236    0.670773
60    -0.300343  166.489777    78.427979  229.551117    0.577561
61  1156.337402  167.820343  1281.515137  228.701355    0.576990
62    76.336815  168.201935   295.248840  227.959595    0.547779

[63 rows x 5 columns]
min_samples =  3
sorted_confident_x = [0, 76, 293, 509, 725, 941, 1157, 1280]
sorted_confident_y = [170, 229, 285, 342, 399, 457, 513, 570, 628, 685]


/tmp/ipykernel_39209/2194842497.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  updated_confident_boxes["x1_upd"][updated_confident_boxes["label_x1"] == x] = x_mean
/tmp/ipykernel_39209/2194842497.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  updated_confident_boxes["x2_upd"][updated_confident_boxes["label_x2"] == x] = x_mean
/tmp/ipykernel_39209/2194842497.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  updated_confid

In [73]:
upd_res

,x1,y1,x2,y2,confidance,label_x1,label_y1,label_x2,label_y2,x1_upd,y1_upd,x2_upd,y2_upd
0,292.159973,286.068909,510.660583,341.723938,0.889665,0,0,2,1,293,285,509,342
1,291.904541,341.950165,510.744263,399.858185,0.885304,0,1,2,5,293,342,509,399
2,292.007660,570.465271,511.048645,627.256409,0.871573,0,2,2,3,293,570,509,628
3,724.143433,630.468567,942.558960,683.702820,0.870791,1,3,4,9,725,628,941,685
4,725.313660,342.440369,942.441467,399.614441,0.870579,1,1,4,5,725,342,941,399
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,939.884033,172.461670,1159.166748,229.938812,0.694375,4,8,6,7,941,170,1157,229
59,507.686584,171.423981,726.961487,231.210236,0.670773,2,8,1,7,509,170,725,229
60,-0.300343,166.489777,78.427979,229.551117,0.577561,5,8,3,7,0,170,76,229
61,1156.337402,167.820343,1281.515137,228.701355,0.576990,6,8,7,7,1157,170,1280,229


In [74]:
show_result_pyplot(model, img_path, [upd_res[["x1_upd", "y1_upd", "x2_upd", "y2_upd", "confidance"]].to_numpy()], score_thr = threshold, out_file = "/home/aiarhipov/centernet/imgs/output.jpg")
